<a href="https://colab.research.google.com/github/HyunWrites0721/LLM_Based_OTT_recommender/blob/main/LLM_Based_OTTRec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os

path = "/content/drive/MyDrive/Colab Notebooks/for_data/ml-1m/"

# 1. Ratings: 유저의 영화 평점 데이터
ratings = pd.read_csv(path + 'ratings.dat', sep='::', engine='python', encoding='latin-1',
                      names=['user_id', 'movie_id', 'rating', 'timestamp'])

# 2. Movies: 영화 제목 및 장르 데이터
movies = pd.read_csv(path + 'movies.dat', sep='::', engine='python', encoding='latin-1',
                     names=['movie_id', 'title', 'genres'])

# 3. Users: 유저 데모그래픽(성별, 나이 등) 데이터
users = pd.read_csv(path + 'users.dat', sep='::', engine='python', encoding='latin-1',
                    names=['user_id', 'gender', 'age', 'occupation', 'zip_code'])

print("데이터 로드 완료!")
print(f"유저 수: {len(users)}, 영화 수: {len(movies)}, 평점 수: {len(ratings)}")

데이터 로드 완료!
유저 수: 6040, 영화 수: 3883, 평점 수: 1000209


In [2]:
# 1. 제목(Title)을 기준으로 고유한 영화 목록 생성
unique_movies = movies.drop_duplicates(subset=['title']).copy()

# 2. 새로운 고유 ID 부여 (0부터 시작)
unique_movies['new_movie_id'] = range(len(unique_movies))

# 3. {기존 ID : 새로운 ID} 매핑 딕셔너리 생성
id_mapping = pd.merge(movies, unique_movies[['title', 'new_movie_id']], on='title')
mapping_dict = dict(zip(id_mapping['movie_id'], id_mapping['new_movie_id']))

# 4. Ratings 데이터 기존 ID를 새로운 고유 ID로 치환 (매핑되지 않는 유령 ID는 제거)
ratings['movie_id'] = ratings['movie_id'].map(mapping_dict)
ratings = ratings.dropna(subset=['movie_id']).astype({'movie_id': int})

# 5. 최종 결과 확인
print(f"기존 영화 수: {len(movies)}")
print(f"정제 후 고유 영화 수: {len(unique_movies)}")

기존 영화 수: 3883
정제 후 고유 영화 수: 3883


In [3]:
unique_movies

,movie_id,title,genres,new_movie_id
0,1,Toy Story (1995),Animation|Children's|Comedy,0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,1
2,3,Grumpier Old Men (1995),Comedy|Romance,2
3,4,Waiting to Exhale (1995),Comedy|Drama,3
4,5,Father of the Bride Part II (1995),Comedy,4
...,...,...,...,...
3878,3948,Meet the Parents (2000),Comedy,3878
3879,3949,Requiem for a Dream (2000),Drama,3879
3880,3950,Tigerland (2000),Drama,3880
3881,3951,Two Family House (2000),Drama,3881


In [4]:
ratings

,user_id,movie_id,rating,timestamp
0,1,1176,5,978300760
1,1,655,3,978302109
2,1,902,3,978301968
3,1,3339,4,978300275
4,1,2286,5,978824291
...,...,...,...,...
1000204,6040,1075,1,956716541
1000205,6040,1078,5,956704887
1000206,6040,558,5,956704746
1000207,6040,1080,4,956715648
